## 6.1 Reading and Writing Data in Text Format
`pandas.read_csv` is one of the most
frequently used in this book.

The optional arguments for these functions may fall into
a few categories:
* *Indexing*
* *Type inference and data conversion*
* *Date and time parsing*
* *Iterating*
* *Unclean data issue*s

In [ ]:
!cat examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

Since this is comma-delimited, we can then use `pandas.read_csv` to read it into a
DataFrame:

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Consider this file:

In [ ]:
!cat examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [ ]:
pd.read_csv("examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
pd.read_csv("examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose you wanted the **message** column to be the index of the returned DataFrame.
You can either indicate you want the column at index 4 or named **"message"** using
the `index_col` argument:

In [ ]:
names = ["a", "b", "c", "d", "message"]
pd.read_csv("examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


If you want to form a hierarchical index from multiple columns, pass a list of column numbers or
names:

In [ ]:
!cat examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [ ]:
parsed = pd.read_csv("examples/csv_mindex.csv", index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some
other pattern to separate fields. Consider a text file that looks like this:

In [ ]:
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


whitespace. In these cases, you can pass a regular expression as a
delimiter for `pandas.read_csv`. This can be expressed by the regular expression `\s+`, so we have then:

In [ ]:
result = pd.read_csv("examples/ex3.txt", sep="\s+")
result

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1673285606.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("examples/ex3.txt", sep="\s+")


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


The file parsing functions have many additional arguments to help you handle the
wide variety of exception file formats that occur

In [ ]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [ ]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Missing data is usually either not present (empty string) or marked by some *sentinel* (placeholder) value. By default, pandas uses a set of commonly
occurring sentinels, such as **NA** and **NULL**:

In [ ]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [ ]:
result = pd.read_csv("examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [ ]:
pd.isna(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


In [ ]:
result = pd.read_csv("examples/ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [ ]:
result2 = pd.read_csv("examples/ex5.csv", keep_default_na=False)
result2

,something,a,b,c,d,message
0,one,1,2,3,4,NA
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [ ]:
result2.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False


In [ ]:
result3 = pd.read_csv("examples/ex5.csv", keep_default_na=False,na_values=["NA"])
result3

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,two,5,6,,8,world
2,three,9,10,11,12,foo


In [ ]:
result3.isna()

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,False,False,False
2,False,False,False,False,False,False


Different NA sentinels can be specified for each column in a dictionary:

In [ ]:
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
pd.read_csv("examples/ex5.csv", na_values=sentinels, keep_default_na=False)

,something,a,b,c,d,message
0,one,1,2,3,4,NaN
1,NaN,5,6,,8,world
2,three,9,10,11,12,NaN


### Reading Text Files in Pieces

Before we look at a large file, we make the pandas display settings more compact:

In [ ]:
pd.options.display.max_rows = 10

In [ ]:
result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If you want to read only a small number of rows (avoiding reading the entire file),
specify that with `nrows`:

In [ ]:
pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read a file in pieces, specify a chunksize as a number of rows:

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

The `TextFileReader` object returned by `pandas.read_csv` allows you to iterate over
the parts of the file according to the chunksize.

In [ ]:
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
tot = pd.Series([], dtype='int64')
for piece in chunker:
  tot = tot.add(piece["key"].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)
tot[:10]

,0
key,
E,368.0
X,364.0
L,346.0
O,343.0
Q,340.0
M,338.0
J,337.0
F,335.0
K,334.0


### Writing Data to Text Format
Data can also be exported to a delimited format. Let’s consider one of the CSV files
read before:

In [ ]:
data = pd.read_csv("examples/ex5.csv")
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame’s `to_csv` method

In [ ]:
data.to_csv("examples/out.csv")

In [ ]:
!cat examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used, of course (writing to **sys.stdout** so it prints the text
result to the console rather than a file):

In [ ]:
import sys
data.to_csv(sys.stdout, sep="|")

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. You might want to denote them
by some other sentinel value:

In [ ]:
data.to_csv(sys.stdout, na_rep="NULL")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


With no other options specified, both the row and column labels are written. Both of
these can be disabled:

In [ ]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [ ]:
data.to_csv(sys.stdout, index=False, columns=["a", "b", "c"])

a,b,c
1,2,3.0
5,6,
9,10,11.0


### Working with Other Delimited Formats
It’s possible to load most forms of tabular data from disk using functions like `pandas.read_csv`. In some cases, however, some manual processing may be necessary.
It’s not uncommon to receive a file with one or more malformed lines that trip up
`pandas.read_csv`. To illustrate the basic tools, consider a small CSV file:

In [ ]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, you can use Python’s built-in csv
module. To use it, pass any open file or file-like object to csv.reader:

In [ ]:
import csv
f = open("examples/ex7.csv")
reader = csv.reader(f)

In [ ]:
for line in reader:
  print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


First, we read the file into a list of lines:

In [ ]:
with open("examples/ex7.csv") as f:
  lines = list(csv.reader(f))

Then we split the lines into the header line and the data lines:

In [ ]:
header, values = lines[0], lines[1:]

Create a dictionary of data columns using a dictionary comprehension
and the expression `zip(*values)`, which transposes rows to columns:

In [ ]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

To define a new format with a different
delimiter, string quoting convention, or line terminator, we could define a simple
subclass of `csv.Dialect`:

In [ ]:
class my_dialect(csv.Dialect):
  lineterminator = "\n"
  delimiter = ";"
  quotechar = '"'
  quoting = csv.QUOTE_MINIMAL

with open("examples/ex7.csv") as f:
  reader = csv.reader(f, dialect=my_dialect)

In [ ]:
with open("examples/ex7.csv") as f:
  reader = csv.reader(f, dialect=my_dialect)

To *write* delimited files manually, you can use `csv.writer`. It accepts an open, writa‐
ble file object and the same dialect and format options as
`csv.reader`:

In [ ]:
with open("mydata.csv", "w") as f:
  writer = csv.writer(f, dialect=my_dialect)
  writer.writerow(("one", "two", "three"))
  writer.writerow(("1", "2", "3"))
  writer.writerow(("4", "5", "6"))
  writer.writerow(("7", "8", "9"))

### JSON Data
JSON (short for JavaScript Object Notation) has become one of the standard formats
for sending data by HTTP request between web browsers and other applications. It
is a much more free-form data format than a tabular text form like CSV.

In [ ]:
obj = """
{"name": "Wes",
  "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
  "pet": null,
  "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
                {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

In [ ]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

`json.dumps`, on the other hand, converts a Python object back to JSON:

In [ ]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other
data structure for analysis will be up to you.

In [ ]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


The `pandas.read_json` can automatically convert JSON datasets in specific arrange‐
ments into a Series or DataFrame.

In [ ]:
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [ ]:
data = pd.read_json("examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


If you need to export data from pandas to JSON, one way is to use the to_json
methods on Series and DataFrame:

In [ ]:
data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [ ]:
data.to_json(sys.stdout, orient="records")

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]

### XML and HTML: Web Scraping
To show how this
works, I downloaded an HTML file (used in the pandas documentation) from the US
FDIC showing bank failures.1 First, you must install some additional libraries used by `read_html`:

    conda install lxml beautifulsoup4 html5lib
If you are not using conda, `pip install lxml` should also work.

The `pandas.read_html` function has a number of options. The result is
a list of DataFrame objects:

In [ ]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)

1

In [ ]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Because **failures** has many columns, pandas inserts a line break character \.

In [ ]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

,count
Closing Date,
2010,157
2009,140
2011,92
2012,51
2008,25
...,...
2004,4
2001,4
2007,3


**Parsing XML with lxml.objectify**

Using `lxml.objectify`, we parse the file and get a reference to the root node of the
XML file with getroot:

In [ ]:
from lxml import objectify
path = "datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
  parsed = objectify.parse(f)
root = parsed.getroot()

`root.INDICATOR` returns a generator yielding each <INDICATOR> XML element. For
each record, we can populate a dictionary of tag names (like YTD_ACTUAL) to data
values (excluding a few tags) by running the following code:

In [ ]:
data = []
skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
                  "DESIRED_CHANGE", "DECIMAL_PLACES"]
for elt in root.INDICATOR:
  el_data = {}
  for child in elt.getchildren():
    if child.tag in skip_fields:
      continue
    el_data[child.tag] = child.pyval
  data.append(el_data)

Lastly, convert this list of dictionaries into a DataFrame:

In [ ]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


pandas’s `pandas.read_xml` function turns this process into a one-line expression:

In [ ]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


## 6.2 Binary Data Formats
One simple way to store (or serialize) data in binary format is using Python’s built-in
`pickle` module. pandas objects all have a `to_pickle` method that writes the data to
disk in pickle format:

In [ ]:
frame = pd.read_csv("examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [ ]:
frame.to_pickle("examples/frame_pickle")

pandas has built-in support for several other open source binary data formats, such
as HDF5, ORC, and Apache Parquet. For example, if you install the pyarrow package
(conda install pyarrow), then you can read Parquet files with pandas.read_par
quet:

In [ ]:
fec = pd.read_parquet('datasets/fec/fec.parquet')

### Reading Microsoft Excel Files
pandas also supports reading tabular data stored in Excel 2003 (and higher) files
using either the `pandas.ExcelFile` class or `pandas.read_excel` function.
These must be installed separately from pandas using
pip or conda:

    conda install openpyxl xlrd

To use `pandas.ExcelFile`, create an instance by passing a path to an xls or xlsx file:

In [ ]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")

In [ ]:
xlsx.sheet_names

['Sheet1']

In [ ]:
xlsx.parse(sheet_name="Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


This Excel table has an index column, so we can indicate that with the `index_col`
argument:

In [ ]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


If you are reading multiple sheets in a file, then it is faster to create the `pandas.Excel`
File, but you can also simply pass the filename to `pandas.read_excel`:

In [ ]:
frame = pd.read_excel("examples/ex1.xlsx", sheet_name="Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


To write pandas data to Excel format, you must first create an ExcelWriter, then
write data to it using the pandas object’s `to_excel` method:

In [ ]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.save()

You can also pass a file path to `to_excel` and avoid the ExcelWriter:

In [ ]:
frame.to_excel("examples/ex2.xlsx")

### Using HDF5 Format

In [ ]:
frame = pd.DataFrame({"a": np.random.standard_normal(100)})
store = pd.HDFStore("examples/mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]
store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [ ]:
store["obj1"]

,a
0,-0.137229
1,0.635939
2,0.238921
3,0.645114
4,0.923305
...,...
95,0.682142
96,-0.807549
97,0.350189
98,-0.696547


HDFStore supports two storage schemas, "fixed" and "table" (the default is
"fixed"). The latter is generally slower, but it supports query operations using a
special syntax:

In [ ]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])

,a
10,0.399031
11,-0.392632
12,1.218760
13,-0.590715
14,-2.098592
15,1.599652


In [ ]:
store.close()

The `pandas.read`_hdf function gives you a shortcut to these tools:

In [ ]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

/tmp/ipython-input-2659235977.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_hdf except for the argument 'path_or_buf' will be keyword-only.
  frame.to_hdf("examples/mydata.h5", "obj3", format="table")


,a
0,-0.137229
1,0.635939
2,0.238921
3,0.645114
4,0.923305


delete the HDF5 file

In [ ]:
import os
os.remove("examples/mydata.h5")

## 6.3 Interacting with Web APIs
Many websites have public APIs providing data feeds via JSON or some other format.
There are a number of ways to access these APIs from Python; one method that I
recommend is the **requests package**, which can be installed with pip or conda:

    conda install requests

To find the last 30 GitHub issues for pandas on GitHub, we can make a GET HTTP
request using the add-on requests library:

In [ ]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp.raise_for_status()
resp

<Response [200]>

It’s a good practice to always call `raise_for_status` after using `requests.get` to
check for HTTP errors.

The response object’s json method will return a Python object containing the parsed
JSON data as a dictionary or list

In [ ]:
data = resp.json()
data[0]["title"]

'BUG: Fix multiindex factorize extension dtypes '

We can pass data directly to pandas.Data
Frame and extract fields of interest:

In [ ]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])
issues

,number,title,labels,state
0,62964,BUG: Fix multiindex factorize extension dtypes,[],open
1,62963,BUG: EngFormatter accuracy wrong,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,62962,DOC: Add versionadded markers for anti joins,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
3,62961,ENH: Add left_semi merge,[],open
4,62960,Fix convert_dtype complex,[],open
...,...,...,...,...
25,62926,BUG: indexing a MultiIndex-ed DataFrame (only)...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
26,62925,BUG: Aligned skew and kurt results with scipy....,[],open
27,62924,DOC: Add filesystem keyword arg and documentat...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
28,62922,BUG: read_parquet fails when passing a list of...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open


## 6.4 Interacting with Databases
The choice of database
is usually dependent on the performance, data integrity, and scalability needs of an
application.

pandas has some functions to simplify loading the results of a SQL query into a
DataFrame.

In [ ]:
import sqlite3
query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20),
        c REAL, d INTEGER
        );"""
con = sqlite3.connect("mydata.sqlite")
con.execute(query)

In [ ]:
con.commit()

Then, insert a few rows of data:

In [ ]:
data = [("Atlanta", "Georgia", 1.25, 6),
        ("Tallahassee", "Florida", 2.6, 3),
        ("Sacramento", "California", 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)

In [ ]:
con.commit()

Most Python SQL drivers return a list of tuples when selecting data from a table:

In [ ]:
cursor = con.execute("SELECT * FROM test")
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [ ]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the
database. The **SQLAlchemy project** is a popular Python SQL toolkit that abstracts
away many of the common differences between SQL databases. pandas has a
read_sql function that enables you to read data easily from a general SQLAlchemy
connection.You can install SQLAlchemy with conda like so:

    conda install sqlalchemy

In [ ]:
import sqlalchemy as sqla
db = sqla.create_engine("sqlite:///mydata.sqlite")
pd.read_sql("SELECT * FROM test", db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


## 6.5 Conclusion
Getting access to data is frequently the first step in the data analysis process. We have
looked at a number of useful tools in this chapter that should help you get started.
In the upcoming chapters we will dig deeper into data wrangling, data visualization,
time series analysis, and other topics.